### For (a), please see find_constricted_set() function
### For (b), please see find_market_equilibrium function

In [4]:
import operator
import copy


class Graph:
    
    def __init__(self, graph):
        #print graph
        self.graph = graph
        self.players = len(graph)
        self.items = len(graph[0])
 
    # A DFS based recursive function that returns true if a matching for u is possible
    def _matching(self, u, matching, visited):
        for v in range(self.items):
            # If player is interest in items v and v is None
            if self.graph[u][v] and visited[v] == False:
                visited[v] = True # Mark v as visited
                #If v is not assigned, or previous player had other choice
                if matching[v] == -1 or self._matching(matching[v], matching, visited):
                    matching[v] = u
                    return True
        return False
 
    # Returns maximum number of matching 
    def max_matching(self):
        #Show if the player was assigned or not.
        matching = [-1]*self.items
        #Count how many items that have been assigned.
        ans = 0
        for i in range(self.players):
            # Mark all items as not visited for next aplayersicant.
            visited = [False] * self.items
            # Find if a player can get a item
            if self._matching(i, matching, visited):
                ans += 1
        return ans, matching
    
    '''
    Returns true if there is a path from source 's' to sink 't' in
    residual graph. Also fills parent[] to store the path
    '''
    def bfs(self, s, t, parent):
 
        # Mark all vertices as not visited
        visited = [False] * (self.players)
        #print visited
         
        # Create a queue for BFS
        queue=[]
         
        # Mark the source node as visited and enqueue it
        queue.append(s)
        visited[s] = True
         
        # Standard BFS Loop
        while queue:
            #Dequeue a vertex from queue and print it
            u = queue.pop(0)
         
            # Get all neighbors of the dequeued vertex u
            # If the neighbor has not been visited, then mark it
            # visited and enqueue it
            for idx, val in enumerate(self.graph[u]):
                if visited[idx] == False and val > 0 :
                    queue.append(idx)
                    visited[idx] = True
                    parent[idx] = u
 
        # If we reached sink in BFS starting from source, then return
        # true, else false
        return True if visited[t] else False
    
    # Returns the maximum flow from s to t in the given graph
    def fordFulkerson(self, source, sink):
 
        # This array is filled by BFS and to store path
        parent = [-1] * (self.players)
        
        max_flow = 0 # There is no flow initially
 
        # Augment the flow while there is path from source to sink
        while self.bfs(source, sink, parent):
 
            # Find minimum residual capacity of the edges along the
            # path filled by BFS. Or we can say find the maximum flow
            # through the path found.
            path_flow = float("Inf")
            s = sink
            while (s != source):
                path_flow = min(path_flow, self.graph[parent[s]][s])
                s = parent[s]
 
            # Add path flow to overall flow
            max_flow += path_flow
 
            # update residual capacities of the edges and reverse edges
            # along the path
            v = sink
            while (v != source):
                u = parent[v]
                self.graph[u][v] -= path_flow
                self.graph[v][u] += path_flow
                v = parent[v]
            print 'parent: ', parent
        #print 'graph after: ', self.graph
        return max_flow, self.graph   

def create_expanded_graph(graph):

    # Construct G' with (1) node s and t, 
    # (2) edges with capacity 1 from s to every node in X
    # (3) edges with capacity 1 from every node in Y to t
    # (4) all edges in original G are set with capacity infinity
    x = len(graph)
    y = len(graph[0])
    n = x + y + 2
    #print '(x, y, n): ', (x, y, n)
    expG = []
    for i in range(n):

        #print 'connection for i: ', i 
        connection = []
        if i == 0: # node s
            # NOT connect s to itself
            connection.append(0)
            # connect s to all nodes in X
            for node in range(x):
                connection.append(1)
            # All nodes in Y is not connected to s
            for node in range(y):
                connection.append(0)
            # s is not connected to t
            connection.append(0) 
        elif i == n - 1: # node t
            # t is not connected to s
            connection.append(0)
            # All nodes in X is not connected to t
            for node in range(x):
                connection.append(0)
            # connect t to all nodes in Y
            for node in range(y):
                connection.append(1)
            # NOT connect t to itself
            connection.append(0)
        elif i >= 1 and i <= x: # nodes in X
            # connect to s
            connection.append(1)
            # NOT connect i in X to all nodes in X
            for node in range(x):
                connection.append(0)
            # connect i in X to nodes in Y according to original graph and set capacity as inf
            for node in range(y):
                if graph[i - 1][node] > 0:
                    connection.append(float('inf'))
                    #connection.append(1)
                else:
                    connection.append(0)
            # NOT connect i in X to node t
            connection.append(0)
        elif i >= x + 1 and i <= 1 + x + y:
            # not connect i in Y to node s
            connection.append(0)
            # connect i in Y to nodes in X according to original graph and set capacity as inf
            for node in range(x):
                connection.append(0)
            # NOT connect i in Y to all nodes in Y
            for node in range(y):
                connection.append(0)
            # connect i in Y to node t
            connection.append(1)

        #print connection
        expG.append(connection)

    return expG

def create_prefered_choice_graph(valuations):

    n = len(valuations) # count of players
    m = len(valuations[0]) # count of items
    prefered_choices = []

    for valuation_player in valuations:
        most_prefered_value = max(valuation_player)
        prefered_choice = []
        for j in range(m):
            if valuation_player[j] == most_prefered_value:
                prefered_choice.append(1)
            else:
                prefered_choice.append(0)

        prefered_choices.append(prefered_choice)

    return prefered_choices

def find_constricted_set(prefered_choice_graph):
        
    expG = create_expanded_graph(prefered_choice_graph)
    #print '\n expG before: ', expG

    expGraph = Graph(expG)
    _, residual_graph = expGraph.fordFulkerson(0, len(expG) - 1)
    print '\n residual_graph: ', residual_graph

    constricted_set = []
    reachable_from_source = []
    residual = Graph(residual_graph)

    for node in range(len(residual_graph)):
        parent = [-1] * (len(residual_graph))
        if residual.bfs(0, node, parent):
            #print node
            reachable_from_source.append(node - 1)

    #print '\n reachable_from_source: ', reachable_from_source

    for node in reachable_from_source:
        if node in range(len(prefered_choice_graph)):
            constricted_set.append(node)

    return constricted_set
    
def find_market_equilibrium(market):

    price = [0 for _ in range(len(market[0]))]

    while True:

        tmp_market = copy.deepcopy(market)
        for buyer in tmp_market:
            for item in range(len(market[0])):
                buyer[item] = buyer[item] - price[item]

        print '\n ==============================================='
        print '\n price: ', price
        print '\n tmp_market: ', tmp_market
        
        prefered_choice_graph = create_prefered_choice_graph(tmp_market)
        print '\n prefered_choice_graph: ', prefered_choice_graph
        constricted_set = find_constricted_set(prefered_choice_graph)
        
        if len(constricted_set) == 0: # there exists a perfect matching
            print "has perfect matching"
            break

        item_to_increase_price = []
        for buyer in constricted_set:
            #print 'buyer in constricted set: ', buyer
            for idx, value in enumerate(prefered_choice_graph[buyer]):
                #print 'idx, value: ', idx, value
                if value > 0 and idx not in item_to_increase_price:
                    item_to_increase_price.append(idx)
        
        print 'item_to_increase_price: ', item_to_increase_price
        for item in item_to_increase_price:
            price[item] += 1
            
        if min(price) > 0:
            price = [item_price - 1 for item_price in price]

        print 'price: ', price

            
    return price, tmp_market
        
        
# market is consisted of buyers, items, and valuation from buyers to items
#market = [[4, 12, 5], [7, 10, 9], [7, 7, 10]]
market = [[1, 1, 1], [1, 1, 1], [1, 0, 1]]
#market = [[1, 1, 0], [1, 1, 0], [1, 0, 0]]
graph = Graph(market)
graph.max_matching()
#find_market_equilibrium(market)




(3, [2, 1, 0])